# Parameter identification example

Here is a simple toy model that we use to demonstrate the working of the inference package

$\emptyset \xrightarrow[]{k_1} X \; \; \; \; X \xrightarrow[]{d_1} \emptyset$

### Run the MCMC algorithm to identify parameters from the experimental data 

In [ ]:
%matplotlib inline
import bioscrape as bs
from bioscrape.types import Model
from bioscrape.simulator import py_simulate_model
from bioscrape.pid_interfaces import *
from bioscrape.emcee_interface import *
import numpy as np
import pylab as plt
import pandas as pd

pid = initialize_mcmc()
# Import bioscrape XML / bioscrape model object M / SBML file
# M  = bs.types.read_model_from_sbml(filename)
pid.M = Model('toy_model.xml')


# Import data from CSV
# Import a CSV file for each experiment run
pid.exp_data = pd.read_csv('test_data.csv', delimiter = '\t', names = ['X','time'], skiprows = 1)
# data_list.append(list(data.get('X')))
# pid.exp_data = np.array(data_list)
pid.measurements = ['X']
pid.time_column = ['time']
# pid.prior = {'k1' : ['gaussian', 10, 100, 1e-7],'d1' : ['gaussian', 0.2, 1, 1e-5]}
pid.prior = {'k1' : ['uniform', 0, 100],'d1' : ['uniform', 0, 50]}
pid.params_to_estimate = ['k1', 'd1']
pid.nwalkers = 200
pid.init_seed = 0.15
pid.nsteps = 3000
pid.type = 'deterministic'


sampler = pid.run_mcmc(plot_show = True)
pid.plot_mcmc_results(sampler);

 58%|█████████████████████████████████████████████▍                                | 1746/3000 [01:10<00:49, 25.27it/s]

In [2]:
np.shape(pid.exp_data['X'])

(50,)

### Check mcmc_results.csv for the results of the MCMC procedure and perform your own analysis.

### You can also plot the results as follows

In [3]:
timepoints = pid.timepoints
M_original = Model('models/toy_model.xml')
sim_x = py_simulate_model(timepoints, Model = M_original, stochastic = False)
sim_x_inf = py_simulate_model(timepoints, Model = fitted_model, stochastic = False)
plt.plot(timepoints, sim_x['X'], 'r--', label = 'Original model')
plt.plot(timepoints, pid.exp_data.get('X'), label = 'Experimental data')
plt.plot(timepoints, sim_x_inf['X'], 'k', label = 'Identified model')

plt.legend()

FileNotFoundError: [Errno 2] No such file or directory: 'models/toy_model.xml'

## Alll methods above have other advanced options that you can use. Refer to Parameter Identification Tools and Advanced Examples notebook for more details. There are many other tools available such as for multiple initial conditions and timepoints for each trajectory, options for the estimator etc.

In [ ]:
fitted_model.get_parameter_values()